<a href="https://colab.research.google.com/github/jihoover77/DS-Unit-2-Linear-Models/blob/master/Linear_Regression2_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

# Step 1:  Read in, wrangle and feature engineer the dataset

In [114]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def wrangle(filepath):

  # Read New York City apartment rental listing data
  df = pd.read_csv(filepath,
                   parse_dates=['created'],
                   index_col='created')
  
  # Remove the most extreme 1% prices,
  # the most extreme .1% latitudes, &
  # the most extreme .1% longitudes
  df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

  # Drop duplicate column
  df.drop(columns='street_address', inplace=True)

  # Engineer a feature called perks_count
  df['perks_count'] = df[['elevator', 'cats_allowed',
                          'hardwood_floors', 'dogs_allowed', 'doorman', 
                          'dishwasher', 'no_fee',
                          'laundry_in_building', 'fitness_center', 'pre-war',
                          'laundry_in_unit',
                          'roof_deck', 'outdoor_space', 'dining_room',
                          'high_speed_internet',
                          'balcony', 'swimming_pool', 'new_construction',
                          'terrace', 'exclusive',
                          'loft', 'garden_patio', 'wheelchair_access',
                          'common_outdoor_space']].sum(axis=1)
  
  # Engineer a feature called total_rooms
  df['bedrooms'] = df['bedrooms'] + 1
  df['bathrooms'] = df['bathrooms'] + 1
  df['total_rooms'] = df['bathrooms'] + df['bedrooms']
  df['bed_bath_ratio'] = df['bedrooms'] / df['bathrooms']

  return df

df = wrangle(DATA_PATH + 'apartments/renthop-nyc.csv')

# Step 2:  Split Data

In [119]:
cutoff = '2016-06-01'
mask = df.index < cutoff
train = df.loc[mask]
test = df.loc[~mask]

target = 'price'
features = ['total_rooms', 'bed_bath_ratio']

y_train = train[target]
X_train = train[features]
y_test = test[target]
X_test = test[features]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((31844, 2), (31844,), (16973, 2), (16973,))

# Step 3:  Establish Baseline

In [120]:
y_pred = [y_train.mean()] * len(y_train)

print('Baseline MAE: ', mean_absolute_error(y_train, y_pred))

Baseline MAE:  1201.8811133682555


# Step 4:  Build Model

In [121]:
# Step 1:  Import predictor class
# Done above

# Step 2:  Instantiate the class
model = LinearRegression()

# Step 3:  Fit the model to training set
model.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Step 5: Check Metrics

In [126]:
y_pred = model.predict(X_test)
print('Linear Model Training MAE: ', mean_absolute_error(y_train, model.predict(X_train)))
print('Linear Model Testing MAE: ', mean_absolute_error(y_test, y_pred))
print('Linear Model Training RMSE: ', mean_squared_error(y_train, model.predict(X_train), squared=False))
print('Linear Model Testing RMSE: ', mean_squared_error(y_test, y_pred, squared=False))
print('Linear Model Training R^2: ', model.score(X_train, y_train))
print('Linear Model Testing R^2: ', model.score(X_test, y_test))

Linear Model Training MAE:  829.3573746257878
Linear Model Testing MAE:  837.6393790333283
Linear Model Training RMSE:  1254.2715535911475
Linear Model Testing RMSE:  1244.0731094487428
Linear Model Training R^2:  0.49333886208081645
Linear Model Testing R^2:  0.5020246747787429


# Step 6:  Communicate the Results

In [131]:
intercept = round(model.intercept_)
coef_1 = round(model.coef_[0])
coef_2 = round(-1 * model.coef_[1])

print(f'PRICE = {intercept} + ({coef_1} * TOTAL ROOMS) - ({coef_2} * BED_BATH_RATIO)')

PRICE = -93 + (1118 * TOTAL ROOMS) - (1418 * BED_BATH_RATIO)


### I added two new features (total rooms and bed bath ratio) to my dataset.  The best score I received was from the mean absolute error $838.  